In [ ]:
from ortools.sat.python import cp_model
import util

In [ ]:
import marimo as mo
import nbformat

# オートマトン制約を用いた数理計画モデル

CP-SAT 固有のオートマトン制約を用いて定式化してみる.

**定数**

- $\Sigma = \lbrace \sigma_1, \dots, \sigma_q \rbrace$: 文字の集合

**決定変数**

- $x_i \in \mathbb{Z}$: 解において $i$ 番目の文字が $\sigma_j$ であれば $j$ となる.
  どの文字も対応しないとき $0$.
  このとき解は $\sigma_{x_0} \sigma_{x_1} \dots$ となる.
  ただし $\sigma_0$ は空文字列とする.
- $v_i \in \lbrace 0, 1 \rbrace$: 解の $i$ 番目が空でないとき $1$. 空のとき $0$.

**制約条件**

- $x_i > 0 \Rightarrow v_i = 1$
- 与えられた文字列 $s \in S$ の各文字を $\Sigma$ を基準にインデックスの配列としたものを $\mathrm{index}_s$ とする.
  $\mathrm{index}_s$ を部分配列にもつすべての配列を受理するオートマトンを文字列の数だけ作成し,
  $\lbrace x_i \rbrace_i$ が受理されるという制約を課す.

**目的関数**

- minimize $\sum_{i} v_i$

In [ ]:
class Model:
    def __init__(self, instance: list[str]):
        max_len = sum(len(s) for s in instance)
        chars = sorted(list(set("".join(instance))))

        cpmodel = cp_model.CpModel()

        cvars = [
            cpmodel.new_int_var(lb=0, ub=len(chars), name="")
            for _ in range(max_len)
        ]

        for s in instance:
            transition_triples = (
                [
                    (idx, jdx + 1, (idx + 1 if c == next_char else idx))
                    for idx, next_char in enumerate(s)
                    for jdx, c in enumerate(chars)
                ]
                + [(idx, 0, idx) for idx, _ in enumerate(s)]
                + [(len(s), 0, len(s))]
                + [(len(s), jdx + 1, len(s)) for jdx, _ in enumerate(chars)]
            )
            cpmodel.add_automaton(
                transition_expressions=cvars,
                starting_state=0,
                final_states=[len(s)],
                transition_triples=transition_triples,
            )

        valids = [cpmodel.new_bool_var("") for _ in cvars]
        for cvar, valid in zip(cvars, valids):
            cpmodel.add(cvar == 0).only_enforce_if(~valid)
        cpmodel.minimize(sum(valids))

        self.instance = instance
        self.chars = chars
        self.cpmodel = cpmodel
        self.cpsolver = cp_model.CpSolver()
        self.cvars = cvars

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        self.cpsolver.parameters.log_search_progress = log
        if time_limit is not None:
            self.cpsolver.parameters.max_time_in_seconds = time_limit
        self.status = self.cpsolver.solve(self.cpmodel)

        return self

    def to_solution(self) -> str | None:
        if self.status not in {cp_model.cp_model_pb2.OPTIMAL, cp_model.cp_model_pb2.FEASIBLE}:
            return None

        solution = ""
        for cvar in self.cvars:
            cidx = self.cpsolver.value(cvar) - 1
            if cidx >= 0:
                solution += self.chars[cidx]

        return solution

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60, log: bool = False) -> str | None:
    return Model(instance).solve(time_limit, log).to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
solution_01 = solve(instance_01)

In [ ]:
_instance = instance_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 63) ---
 Sol: ulcinycostkgjiqfeovoazppglnbrdxdxbcsvrvsnkuhmpxnhtqpgxzvxissbxf
str1: ---------tkg--------------n--------------kuhmpxnhtq-gxzvxis----
str2: ---i---o----jiqf-o-------lnb--x-x-c-v--s--u-------qp---v-issbxf
str3: ulcinycos--------ovo-zpp-l-------------------p-----------------
str4: ---i-------g----e-v-az--g--brd-d-bcsvrv-n------n----g---------f

solution is feasible: True


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
solution_02 = solve(instance_02)

In [ ]:
_instance = instance_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 135) ---
 Sol: tkgnkuhmpxnhtqgiojiqfolnbxxcvsuqpvlcingesycosovazgbrddbcsvoplrzxucpmqvnngtdfeuivdcvdpfzsmsbroqvnbczfjtvxerzbrvhiwgpxqkrdrlctodtmprpxwed
str1: tkgnkuhmpxnhtqg----------x----------------------z--------v-----x--------------i--------s-----------------------------------------------
str2: ---------------iojiqfolnbxxcvsuqpv--i---s---s-----b------------x-----------f-----------------------------------------------------------
str3: -----u----------------l----c--------in---ycosov-----------o---z---p-----------------p------------------------------------l------p------
str4: ---------------i----------------------ge------vazgbrddbcsv---r-------vnng--f---------------------------------------------------

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
solution_03 = solve(instance_03)

In [ ]:
_instance = instance_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 242) ---
  Sol: tkgnkuhmpxnhiojiqfolnbxxulcinycosoigevoazgbrddbcsvpyplrzxucpmqvgtdfupbdevdcvdpfzsmsenbczfjtvxerzbrvigxwxqkrdrlctodtmpkkqafigqjwokkskrlxxpabivbvzkozlfsavnqaxudgqvqcewbfgijowwyrsxqjnfpadiusiqbwezshkvhcomiuvddhtxxqjzqbctbakxusfcfzpeecvwantfmgqzu
str01: tkgnkuhmpxnh----------------------------------------------------t---------------------------------------q------------------g----------x--------z-------v---x------------i------s------------------------------------------------

In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
solution_04 = solve(instance_04)

In [ ]:
_instance = instance_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 28) ---
  Sol: baeddcbacdeeaecbdbceabdcbade
str01: ---d-cb-c-----c-dbc----c---e
str02: b--dd-b---ee-e-----e-bd-----
str03: -----c-acdee--c----e-b-----e
str04: -aedd----d------d--e-bd---d-
str05: -a---cb---ee--c-----ab-c---e
str06: b-----ba-------b---e-bdcba--
str07: b-----ba--e-ae-b----a-d--a--
str08: --e-------ee-ecbdb-e-------e
str09: -----c--cdee----d---a-dc--d-
str10: b--d---a-------bdb-ea----ad-

solution is feasible: True


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
solution_05 = solve(instance_05)

In [ ]:
_instance = instance_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 39) ---
  Sol: dacebaedacebabcdbaedcbcaedb

In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
solution_06 = solve(instance_06)

In [ ]:
_instance = instance_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: TATAGGCACGTAAGTCATCGACTA
str01: -AT-GG---G-A--T-A-CG----
str02: -ATA--C-C-T---TC--C--C--
str03: ------CACG-AA-T--T-GA---
str04: TA-A---A---A--TC-T-G--T-
str05: -A--GG----TAA--CA---A--A
str06: T-T---C-C-TA-G-----G--TA
str07: T-T-G-----TA-G--ATC---T-
str08: T---GG---G-AAGT--TC-----
str09: T-T---C-C--A---CA---ACT-
str10: T-----C---TAA---A-CGA--A

solution is feasible: True


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
solution_07 = solve(instance_07)

In [ ]:
_instance = instance_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
solution_08 = solve(instance_08)

In [ ]:
_instance = instance_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 46) ---
  Sol: MEQSKFPTRNSALSEYQNFSRDNAVIHACPGVFRKGTANEGLYHDQ
str01: M----------ALS-Y------------CP----KGT---------
str02: M-QS------S-L----N-----A-I---P-V--------------
str03: M-----P-----LS-YQ---------H-----FRK-----------
str04: ME------------E-----------H----V------NE-L-HD-
str05: M--S-----N--------F--D-A-I-------R---A---L----
str06: M----F--RN------QN-SR-N-------G---------------
str07: M----F---------Y-------A--HA----F--G----G-Y---
str08: M--SKF-TR-----------R--------P------------Y--Q
str09: M--S-F------------------V--A--GV----TA-------Q
str10: ME-S--------L-----------V----PG-F-----NE------

solution is feasible: True


In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
solution_09 = solve(instance_09)    

In [ ]:
_instance = instance_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS